## Created by <a href="https://github.com/yunsuxiaozi">yunsuxiaozi</a> 2024/5/2

### 感知机算法说的简单点就是用线性回归做分类,比如做二分类任务,先给数据打上1和-1的标签,然后用线性回归来拟合,大于0的数据分为1这类,小于0的分为-1这类.关于线性回归的介绍可以看这篇文章<a href="https://www.kaggle.com/code/yunsuxiaozi/linear-regression-algorithm-from-scratch">Linear Regression algorithm from scratch</a>

### 这里选择的数据集是鸢尾花数据集<a href="https://www.kaggle.com/datasets/uciml/iris">Iris</a>.

In [1]:
import pandas as pd#处理表格数据的库
import numpy as np#进行矩阵运算的库
import random#提供了一些用于生成随机数的函数
#设置随机种子,保证模型可以复现
def seed_everything(seed):
    np.random.seed(seed)#numpy的随机种子
    random.seed(seed)#python内置的随机种子
seed_everything(seed=2024)

In [2]:
dataset=pd.read_csv("/kaggle/input/iris/Iris.csv")
print(f"len(dataset):{len(dataset)}")
dataset.drop(['Id'],axis=1,inplace=True)
#将label打上1和-1的标签
dataset['Species']=(dataset['Species']=='Iris-setosa').astype(int)*2-1
dataset.head()

len(dataset):150


,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,1
1,4.9,3.0,1.4,0.2,1
2,4.7,3.2,1.3,0.2,1
3,4.6,3.1,1.5,0.2,1
4,5.0,3.6,1.4,0.2,1


In [3]:
X=dataset.drop(['Species'],axis=1).values
y=dataset['Species'].values

In [4]:
class Perceptron():
    def __init__(self,n_iterators=50,batch_size=100000,lr=0.01):
        self.n_iterators=n_iterators#迭代多少次
        self.batch_size=batch_size#用迭代算法每次传入的批次数量
        self.lr=lr#学习率
        
    def fit(self,train_X,train_y):
        """
        train_X.shape:(num_data,num_features)
        train_y.shape:(num_data,y.shape[1])
        self.W.shape:(num_features,y.shape[1])
        self.b.shape:(1,y.shape[1])
        """
        if len(train_y.shape)==1:
            train_y=train_y.reshape(-1,1)
        self.W=np.random.randn(train_X.shape[1],train_y.shape[1])
        self.b=np.random.randn(1,train_y.shape[1])
        for iterator in range(self.n_iterators):  
            random_number=np.arange(len(train_X))
            np.random.shuffle(random_number)
            train_X=train_X[random_number]
            train_y=train_y[random_number]
            for idx in range(0,len(train_X),self.batch_size):
                train_X1=train_X[idx:idx+self.batch_size]
                train_y1=train_y[idx:idx+self.batch_size]
                train_pred=train_X1.dot(self.W)+self.b
                temp_b=np.tile(self.b,(train_X1.shape[0],1))
                #更新参数
                self.W=self.W-2*self.lr*(train_X1.T).dot(train_pred-train_y1)/len(train_X1)
                self.b=self.b-2*self.lr*np.mean(train_pred-train_y1,axis=0)

    def predict(self,test_X):
        test_pred=np.zeros((len(test_X),self.b.shape[1]))
        for idx in range(0,len(test_X),self.batch_size):
            test_X1=test_X[idx:idx+self.batch_size]
            test_pred[idx:idx+len(test_X1)]=test_X1.dot(self.W)+self.b
        return np.where(test_pred.reshape(-1)>=0,1,-1)

In [5]:
model=Perceptron()
model.fit(X,y)
y_pred=model.predict(X)
print(f"accuracy:{np.mean(y==y_pred)}")

accuracy:1.0
